In [3]:
from prompt_templates import TASK_PROMPT
import pandas as pd


df = pd.read_csv("../data/eval/dependencies_without_rules.csv", sep=";")

for x in df.to_dict("records"):
    print(TASK_PROMPT.format(
        nameA=x["option_name"],
        typeA=x["option_type"],
        valueA=x["option_value"],
        fileA=x["option_file"],
        technologyA=x["option_technology"],
        nameB=x["dependent_option_name"],
        typeB=x["dependent_option_type"],
        valueB=x["dependent_option_value"],
        fileB=x["dependent_option_file"],
        technologyB=x["dependent_option_technology"],
    ))


Carefully evaluate whether configuration option spring.datasource.druid.stat-view-servlet.login-password of type ConfigType.PASSWORD with value druid in mall-admin/src/main/resources/application-dev.yml of technology spring     depends on configuration option spring.datasource.druid.stat-view-servlet.login-password of type ConfigType.PASSWORD with value druid in mall-search/src/main/resources/application-prod.yml of technology spring or vice versa.
Carefully evaluate whether configuration option project.groupId of type ConfigType.NAME with value com.macro.mall in mall-mbg/pom.xml of technology maven     depends on configuration option project.parent_mall.groupId of type ConfigType.NAME with value com.macro.mall in mall-mbg/pom.xml of technology maven or vice versa.
Carefully evaluate whether configuration option spring.h2.console.enabled of type ConfigType.BOOLEAN with value true in apollo-biz/src/test/resources/application.properties of technology spring     depends on configuration o

In [4]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env")

pinecone_client = Pinecone(api_key=os.getenv(key="PINECONE_API_KEY"))

#index = pinecone_client.Index("github")

#if index:
#    print(index.describe_index_stats())
#    print(index.describe_index_stats()["total_vector_count"])

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
from ingestion import IngestionEngine

engine = IngestionEngine(
    pinecone_client=pinecone_client,
    splitting="sentence",
    extractors=[]
)

docs = engine.docs_from_github(project_name="apollo")

print(len(docs))

[06/28/24 12:48:51] INFO     Ingestion engine initialized.                                          ]8;id=985342;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=216985;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     Start scraping the repository of apollo.                              ]8;id=28137;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=438315;file:///home/simisimon/GitHub/cval/src/ingestion.py#180\180]8;;\

In [1]:
from plugins.prompt_engine_factory import PromptEngineFactory
from data import Dependency

dep = Dependency(
    project="piggymetrics",
    dependency_category="value-equality",
    option_name="EXPOSE",
    option_value="8080",
    option_type="PORT",
    option_file="Dockerfile",
    option_technology="Docker",
    dependent_option_name="server.port",
    dependent_option_value="8080",
    dependent_option_file="application.yml",
    dependent_option_type="PORT",
    dependent_option_technology="Spring-Boot"
)


prompt_engine = PromptEngineFactory().get_prompt_engine(tool_name="cfgnet")

prompt_engine.get_task_str(dependency=dep)

'Carefully evaluate whether configuration option EXPOSE of type PORT with value 8080 in Dockerfile of technology Dockerdepends on configuration option server.port of type PORT with value 8080 in application.yml of technology Spring-Boot or vice versa.'